In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
df=pd.read_csv('FlightDelays.csv')
m=len(df)
#making time continuous 
q=df['DEP_TIME']
p=df['CRS_DEP_TIME']
a=p
for i in range(m):
    a[i]=int(p[i]/100)*60+p[i]-int(p[i]/100)*100
b=q
for i in range(m):
    b[i]=int(q[i]/100)*60+q[i]-int(q[i]/100)*100
df['DEP_TIME']=b
df['CRS_DEP_TIME']=a



df.loc[(df.DEP_TIME <= 100),'DEP_TIME']+=1440

#indexNames = df[ df['DEP_TIME'] <= 100 ].index
# Delete these row indexes from dataFrame
#df.drop(indexNames , inplace=True)
print(df.head())
df=df.sample(frac=1,random_state=42);
print(df.describe())
print(df['FL_NUM'].nunique())
print(df['TAIL_NUM'].nunique())
#from info method we can observe that there are no null values in the given dataset
#from heatmap of correlation matrix we can infer there is good corelation between CRS_DEP_TIME and DEP_TIME
#from describe method we can infer that the delays caused by weather conditions are less,number of samples are 2201\

olddf=df

print(df.info())
print(df.describe())

In [ ]:
df.groupby(['Weather'])['Flight Status'].value_counts(normalize=True)
df.groupby(['ORIGIN'])['Flight Status'].value_counts(normalize=True)
df.groupby(['CARRIER'])['Flight Status'].value_counts(normalize=True)

In [ ]:
sn.heatmap(df.corr())

y=df.loc[:,'Flight Status']
df['year'] = pd.DatetimeIndex(df['FL_DATE']).year
df['month'] = pd.DatetimeIndex(df['FL_DATE']).month
df=df.drop(columns=['FL_DATE','Flight Status'])
for column in df.columns:
    if df[column].dtype==object:
        dummyCols=pd.get_dummies(df[column])
        df=df.join(dummyCols)
        del df[column]
print(df.head())        
y=y.replace(['ontime','delayed'],[0,1])
print(y)

y.describe()


In [ ]:
import matplotlib
z = df['DEP_TIME']-df['CRS_DEP_TIME']
x=df['CRS_DEP_TIME']
label = y
colors = ['green','red']
fig = plt.figure(figsize=(4,4))
plt.scatter(x, z, c=label, cmap=matplotlib.colors.ListedColormap(colors))
plt.xlabel('crs_dep_time')
plt.ylabel('difference in time')


z = df['Weather']
x=df['DEP_TIME']
label = y
colors = ['green','red']
fig = plt.figure(figsize=(4,4))
plt.scatter(x, z, c=label, cmap=matplotlib.colors.ListedColormap(colors))
plt.xlabel('dep_time')
plt.ylabel('weather')


z = df['DISTANCE']
x=df['DEP_TIME']
label = y
colors = ['green','red']

fig = plt.figure(figsize=(4,4))
plt.scatter(x, z, c=label, cmap=matplotlib.colors.ListedColormap(colors))
plt.xlabel('dep_time')
plt.ylabel('distance')

#from the scatter plot of dep_time-crs_dep_time vs crs_dep_time one can clearly see that the flight delayed 
#when difference is more .So the difference is one of the features. also seems like there are 2 outliers which are to be taken care of.  


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression(C=0.01,max_iter=500)

X=df;
X_train,X_test,y_train,y_test=train_test_split(df,y,test_size=0.4,random_state=42)
log_reg.fit(X_train,y_train)
y_pred=log_reg.predict(X_test);

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred))
print(log_reg.coef_)

In [ ]:
#from the coefficients it is very clear that crs_dep_time , dep_time , weather are the most important features 
#the dummy features created from flight numbers and tail numbers are least important 
#due to these large number of unnecessary dummy variables it seems that the accuracy achieved is lower 
#Logistic Regression is the right model for this problem, as the problem is a binary classification problem.
#SVM, KNN ,Decision tree algorithms can also be used but as training data is less, there are only two outliers, after removal of 
#irrelevant features there are only 3 categorical features, significantly more number of features 
#implies that Logistic Regression is best choice.

In [ ]:
#As stated earlier flight number, month , year , tail number are not the relavant feature removing them from feature set
df=olddf;
df=df.drop(columns=['FL_DATE','Flight Status','FL_NUM','TAIL_NUM'])
for column in df.columns:
    if df[column].dtype==object:
        dummyCols=pd.get_dummies(df[column])
        df=df.join(dummyCols)
        del df[column]
print(df.head())  



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression(C=0.01,max_iter=500)
scaler=StandardScaler()
X=df;

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=42)
log_reg.fit(X_train,y_train)
y_pred=log_reg.predict(X_test);

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))
print(y_test.value_counts())
print(f1_score(y_test, y_pred))
print(log_reg.coef_)